In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import json

d:\дз\TerrariaRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../data/wiki_dump_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


inputs = [item["content"] for item in data if "content" in item]

In [3]:
documents = [Document(page_content=text) for text in inputs]

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=1000, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)

In [4]:
len(chunks)

11161

In [5]:
# не запускать если все плохо с гпу, либо нет cuda
# если нет cuda качать так: 
# pip uninstall torch torchvision torchaudio -y
# pip install torch --index-url https://download.pytorch.org/whl/cu118 --no-cache-dir --no-deps
#
# проверить
# import torch
# print(torch.__version__)
# print(torch.cuda.is_available())


embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": "cuda"}
)

In [1]:
# если уже есть база, то загрузить ее так:
# vectorstore = Chroma(persist_directory="./terraria_db", embedding_function=embeddings)

In [6]:
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./terraria_db")
retriever = vectorstore.as_retriever(search_kwargs={"k": 7})

In [14]:
query = "Как найти Королеву пчёл?"

results = retriever._get_relevant_documents(query, run_manager=None)

for i, doc in enumerate(results):
    #print(f"Document {i+1}: similarity = {score:.4f}")
    print(doc.page_content)
    print("---------------------------------------------------------------------------------------------------------")

'''Королева пчёл''' (Королева пчёл)— это босс простого режима. Призывается при уничтожении личинки, появляющейся внутри пчелиных ульев, расположенных в подземных джунглей, или с помощью пчелиной мерзости. Победа над королевой пчёл необязательна для прохождения игры, однако с неё выпадают полезные предметы и оружия, а также победа над ней позволит шаману заселиться при наличии свободного жилья.
== Поведение ==
'''Королева пчёл''' имеет несколько видов атак:'''
* Королева пчёл выпрямляется, а затем трижды летает взад и вперед, пытаясь протаранить игрока. Она всегда будет использовать эту атаку первой.
* Королева пчёл летает на месте и рожает множество маленьких пчёл (В режиме эксперта и мастера ещё и обычных пчёл)
---------------------------------------------------------------------------------------------------------
** Также если правильно рассчитать момент тарана щитом Ктулху, то можно заблокировать весь урон от этой атаки.
* королеву пчёл желательно убивать в подземных джунглях, так 